In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
;; A square is represented as a cons of (row . col), both naturals 0-7
(defun square-p (sq)
  (declare (xargs :guard t))
  (and (consp sq)
       (natp (car sq)) (< (car sq) 8)
       (natp (cdr sq)) (< (cdr sq) 8)))


Since SQUARE-P is non-recursive, its admission is trivial.  We observe
that the type of SQUARE-P is described by the theorem 
(OR (EQUAL (SQUARE-P SQ) T) (EQUAL (SQUARE-P SQ) NIL)).  

Computing the guard conjecture for SQUARE-P....

The guard conjecture for SQUARE-P is trivial to prove, given the :compound-
recognizer rule NATP-COMPOUND-RECOGNIZER.  SQUARE-P is compliant with
Common Lisp.

Summary
Form:  ( DEFUN SQUARE-P ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SQUARE-P


In [3]:
;; Piece types (simplified)
(defun piece-type-p (x)
  (declare (xargs :guard t))
  (member-equal x '(:king :queen :rook :bishop :knight :pawn)))


Since PIECE-TYPE-P is non-recursive, its admission is trivial.  We
observe that the type of PIECE-TYPE-P is described by the theorem 
(OR (CONSP (PIECE-TYPE-P X)) (EQUAL (PIECE-TYPE-P X) NIL)).  We used
the :type-prescription rule MEMBER-EQUAL.

Computing the guard conjecture for PIECE-TYPE-P....

The guard conjecture for PIECE-TYPE-P is trivial to prove, given the
:executable-counterpart of TRUE-LISTP.  PIECE-TYPE-P is compliant with
Common Lisp.

Summary
Form:  ( DEFUN PIECE-TYPE-P ...)
Rules: ((:EXECUTABLE-COUNTERPART TRUE-LISTP)
        (:TYPE-PRESCRIPTION MEMBER-EQUAL))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 PIECE-TYPE-P


In [4]:
;; A piece is (type . square)
(defun piece-p (p)
  (declare (xargs :guard t))
  (and (consp p)
       (piece-type-p (car p))
       (square-p (cdr p))))


Since PIECE-P is non-recursive, its admission is trivial.  We observe
that the type of PIECE-P is described by the theorem 
(OR (EQUAL (PIECE-P P) T) (EQUAL (PIECE-P P) NIL)).  We used the :type-
prescription rule SQUARE-P.

Computing the guard conjecture for PIECE-P....

The guard conjecture for PIECE-P is trivial to prove.  PIECE-P is compliant
with Common Lisp.

Summary
Form:  ( DEFUN PIECE-P ...)
Rules: ((:TYPE-PRESCRIPTION SQUARE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PIECE-P


In [5]:
(defun piece-type (p)
  (declare (xargs :guard (piece-p p)))
  (car p))


Since PIECE-TYPE is non-recursive, its admission is trivial.  We could
deduce no constraints on the type of PIECE-TYPE.

Computing the guard conjecture for PIECE-TYPE....

The non-trivial part of the guard conjecture for PIECE-TYPE is

Goal
(IMPLIES (AND (PIECE-P P) (NOT (CONSP P)))
         (EQUAL P NIL)).

Q.E.D.

That completes the proof of the guard theorem for PIECE-TYPE.  PIECE-TYPE
is compliant with Common Lisp.

Summary
Form:  ( DEFUN PIECE-TYPE ...)
Rules: ((:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION SQUARE-P))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Prover steps counted:  89
 PIECE-TYPE


In [6]:
(defun piece-square (p)
  (declare (xargs :guard (piece-p p)))
  (cdr p))


Since PIECE-SQUARE is non-recursive, its admission is trivial.  We
could deduce no constraints on the type of PIECE-SQUARE.

Computing the guard conjecture for PIECE-SQUARE....

The non-trivial part of the guard conjecture for PIECE-SQUARE is

Goal
(IMPLIES (AND (PIECE-P P) (NOT (CONSP P)))
         (EQUAL P NIL)).

Q.E.D.

That completes the proof of the guard theorem for PIECE-SQUARE.  PIECE-SQUARE
is compliant with Common Lisp.

Summary
Form:  ( DEFUN PIECE-SQUARE ...)
Rules: ((:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION SQUARE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  89
 PIECE-SQUARE


In [7]:
;; Abstract predicate: does piece P attack square SQ?
;; (In a full implementation, this would encode piece movement rules)
(defstub attacks (piece sq) t)


Summary
Form:  (DEFSTUB ATTACKS ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 ATTACKS


In [8]:
;; A position contains: our king, attacker(s), and other pieces
;; Simplified: just track the key elements for the proof
(defun position-p (pos)
  (declare (xargs :guard t))
  (and (consp pos)
       (piece-p (car pos))                    ; our-king
       (eql (piece-type (car pos)) :king)
       (consp (cdr pos))
       (piece-p (cadr pos))                   ; attacker
       (true-listp (cddr pos))))              ; other-pieces (blockers)


Since POSITION-P is non-recursive, its admission is trivial.  We observe
that the type of POSITION-P is described by the theorem 
(OR (EQUAL (POSITION-P POS) T) (EQUAL (POSITION-P POS) NIL)).  

Computing the guard conjecture for POSITION-P....

The guard conjecture for POSITION-P is trivial to prove, given the
:executable-counterpart of EQLABLEP.  POSITION-P is compliant with
Common Lisp.

Summary
Form:  ( DEFUN POSITION-P ...)
Rules: ((:EXECUTABLE-COUNTERPART EQLABLEP))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 POSITION-P


In [9]:
(defun our-king (pos)
  (declare (xargs :guard (position-p pos)))
  (car pos))


Since OUR-KING is non-recursive, its admission is trivial.  We could
deduce no constraints on the type of OUR-KING.

Computing the guard conjecture for OUR-KING....

The non-trivial part of the guard conjecture for OUR-KING is

Goal
(IMPLIES (AND (POSITION-P POS) (NOT (CONSP POS)))
         (EQUAL POS NIL)).

Q.E.D.

That completes the proof of the guard theorem for OUR-KING.  OUR-KING
is compliant with Common Lisp.

Summary
Form:  ( DEFUN OUR-KING ...)
Rules: ((:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION POSITION-P)
        (:DEFINITION SQUARE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  230
 OUR-KING


In [10]:
(defun attacker (pos)
  (declare (xargs :guard (position-p pos)))
  (cadr pos))


Since ATTACKER is non-recursive, its admission is trivial.  We could
deduce no constraints on the type of ATTACKER.

Computing the guard conjecture for ATTACKER....

The non-trivial part of the guard conjecture for ATTACKER is

Goal
(AND (IMPLIES (AND (POSITION-P POS) (NOT (CONSP POS)))
              (EQUAL POS NIL))
     (IMPLIES (AND (POSITION-P POS)
                   (NOT (CONSP (CDR POS))))
              (EQUAL (CDR POS) NIL))).

Q.E.D.

That completes the proof of the guard theorem for ATTACKER.  ATTACKER
is compliant with Common Lisp.

Summary
Form:  ( DEFUN ATTACKER ...)
Rules: ((:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION POSITION-P)
        (:DEFINITION SQUARE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  464
 ATTACKER


In [11]:
(defun other-pieces (pos)
  (declare (xargs :guard (position-p pos)))
  (cddr pos))


Since OTHER-PIECES is non-recursive, its admission is trivial.  We
could deduce no constraints on the type of OTHER-PIECES.

Computing the guard conjecture for OTHER-PIECES....

The non-trivial part of the guard conjecture for OTHER-PIECES is

Goal
(AND (IMPLIES (AND (POSITION-P POS) (NOT (CONSP POS)))
              (EQUAL POS NIL))
     (IMPLIES (AND (POSITION-P POS)
                   (NOT (CONSP (CDR POS))))
              (EQUAL (CDR POS) NIL))).

Q.E.D.

That completes the proof of the guard theorem for OTHER-PIECES.  OTHER-PIECES
is compliant with Common Lisp.

Summary
Form:  ( DEFUN OTHER-PIECES ...)
Rules: ((:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION POSITION-P)
        (:DEFINITION SQUARE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  464
 OTHER-PIECES


In [12]:
;; The king is in check if the attacker attacks the king's square
(defun in-check-p (pos)
  (declare (xargs :guard (position-p pos)))
  (attacks (attacker pos) (piece-square (our-king pos))))


Since IN-CHECK-P is non-recursive, its admission is trivial.  We could
deduce no constraints on the type of IN-CHECK-P.

Computing the guard conjecture for IN-CHECK-P....

The non-trivial part of the guard conjecture for IN-CHECK-P is

Goal
(IMPLIES (POSITION-P POS)
         (PIECE-P (OUR-KING POS))).
Goal'

Q.E.D.

That completes the proof of the guard theorem for IN-CHECK-P.  IN-CHECK-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN IN-CHECK-P ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:DEFINITION MEMBER-EQUAL)
        (:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION OUR-KING)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION POSITION-P)
        (:DEFINITION SQUARE-P)
        (:EXECUTABLE-COUNTERPART CAR)
        (:EXECUTABLE-COUNTERPART CDR)
        (:EXECUTABLE-COUNTERPART CONSP)
        (:EXECUTABLE-COUNTERPART MEMBER-EQUAL)
        (:EXECUTABLE-COUNTERPART NOT

In [13]:
;; Move type enumeration
(defun move-type-p (x)
  (declare (xargs :guard t))
  (member-equal x '(:king-move :block :capture)))


Since MOVE-TYPE-P is non-recursive, its admission is trivial.  We observe
that the type of MOVE-TYPE-P is described by the theorem 
(OR (CONSP (MOVE-TYPE-P X)) (EQUAL (MOVE-TYPE-P X) NIL)).  We used
the :type-prescription rule MEMBER-EQUAL.

Computing the guard conjecture for MOVE-TYPE-P....

The guard conjecture for MOVE-TYPE-P is trivial to prove, given the
:executable-counterpart of TRUE-LISTP.  MOVE-TYPE-P is compliant with
Common Lisp.

Summary
Form:  ( DEFUN MOVE-TYPE-P ...)
Rules: ((:EXECUTABLE-COUNTERPART TRUE-LISTP)
        (:TYPE-PRESCRIPTION MEMBER-EQUAL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 MOVE-TYPE-P


In [14]:
;; A move specifies: which piece moves, from where, to where, and type
(defun move-p (m)
  (declare (xargs :guard t))
  (and (consp m)
       (piece-p (car m))              ; piece that moves
       (consp (cdr m))
       (square-p (cadr m))            ; destination square
       (consp (cddr m))
       (move-type-p (caddr m))))      ; move type


Since MOVE-P is non-recursive, its admission is trivial.  We observe
that the type of MOVE-P is described by the theorem 
(OR (CONSP (MOVE-P M)) (EQUAL (MOVE-P M) NIL)).  We used the :type-
prescription rule MOVE-TYPE-P.

Computing the guard conjecture for MOVE-P....

The guard conjecture for MOVE-P is trivial to prove.  MOVE-P is compliant
with Common Lisp.

Summary
Form:  ( DEFUN MOVE-P ...)
Rules: ((:TYPE-PRESCRIPTION MOVE-TYPE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 MOVE-P


In [15]:
(defun move-piece (m)
  (declare (xargs :guard (move-p m)))
  (car m))


Since MOVE-PIECE is non-recursive, its admission is trivial.  We could
deduce no constraints on the type of MOVE-PIECE.

Computing the guard conjecture for MOVE-PIECE....

The non-trivial part of the guard conjecture for MOVE-PIECE is

Goal
(IMPLIES (AND (MOVE-P M) (NOT (CONSP M)))
         (EQUAL M NIL)).

Q.E.D.

That completes the proof of the guard theorem for MOVE-PIECE.  MOVE-PIECE
is compliant with Common Lisp.

Summary
Form:  ( DEFUN MOVE-PIECE ...)
Rules: ((:DEFINITION MOVE-P)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION SQUARE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  201
 MOVE-PIECE


In [16]:
(defun move-dest (m)
  (declare (xargs :guard (move-p m)))
  (cadr m))


Since MOVE-DEST is non-recursive, its admission is trivial.  We could
deduce no constraints on the type of MOVE-DEST.

Computing the guard conjecture for MOVE-DEST....

The non-trivial part of the guard conjecture for MOVE-DEST is

Goal
(AND (IMPLIES (AND (MOVE-P M) (NOT (CONSP M)))
              (EQUAL M NIL))
     (IMPLIES (AND (MOVE-P M) (NOT (CONSP (CDR M))))
              (EQUAL (CDR M) NIL))).

Q.E.D.

That completes the proof of the guard theorem for MOVE-DEST.  MOVE-DEST
is compliant with Common Lisp.

Summary
Form:  ( DEFUN MOVE-DEST ...)
Rules: ((:DEFINITION MOVE-P)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION SQUARE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  406
 MOVE-DEST


In [17]:
(defun move-type (m)
  (declare (xargs :guard (move-p m)))
  (caddr m))


Since MOVE-TYPE is non-recursive, its admission is trivial.  We could
deduce no constraints on the type of MOVE-TYPE.

Computing the guard conjecture for MOVE-TYPE....

The non-trivial part of the guard conjecture for MOVE-TYPE is

Goal
(AND (IMPLIES (AND (MOVE-P M) (NOT (CONSP M)))
              (EQUAL M NIL))
     (IMPLIES (AND (MOVE-P M) (NOT (CONSP (CDDR M))))
              (EQUAL (CDDR M) NIL))
     (IMPLIES (AND (MOVE-P M) (NOT (CONSP (CDR M))))
              (EQUAL (CDR M) NIL))).

Q.E.D.

That completes the proof of the guard theorem for MOVE-TYPE.  MOVE-TYPE
is compliant with Common Lisp.

Summary
Form:  ( DEFUN MOVE-TYPE ...)
Rules: ((:DEFINITION MOVE-P)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION SQUARE-P))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  613
 MOVE-TYPE


In [18]:
;; A move is a king-move if the king itself moves
(defun is-king-move-p (m pos)
  (declare (xargs :guard (and (move-p m) (position-p pos))))
  (equal (piece-square (move-piece m))
         (piece-square (our-king pos))))


Since IS-KING-MOVE-P is non-recursive, its admission is trivial.  We
observe that the type of IS-KING-MOVE-P is described by the theorem
(OR (EQUAL (IS-KING-MOVE-P M POS) T) (EQUAL (IS-KING-MOVE-P M POS) NIL)).
We used primitive type reasoning.

Computing the guard conjecture for IS-KING-MOVE-P....

The non-trivial part of the guard conjecture for IS-KING-MOVE-P is

Goal
(AND (IMPLIES (AND (POSITION-P POS) (MOVE-P M))
              (PIECE-P (MOVE-PIECE M)))
     (IMPLIES (AND (POSITION-P POS) (MOVE-P M))
              (PIECE-P (OUR-KING POS)))).
Subgoal 2
Subgoal 1

Q.E.D.

That completes the proof of the guard theorem for IS-KING-MOVE-P. 
IS-KING-MOVE-P is compliant with Common Lisp.

Summary
Form:  ( DEFUN IS-KING-MOVE-P ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:DEFINITION MEMBER-EQUAL)
        (:DEFINITION MOVE-P)
        (:DEFINITION MOVE-PIECE)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION 

In [19]:
;; A move is a capture if the destination is the attacker's square
(defun is-capture-p (m pos)
  (declare (xargs :guard (and (move-p m) (position-p pos))))
  (equal (move-dest m)
         (piece-square (attacker pos))))


Since IS-CAPTURE-P is non-recursive, its admission is trivial.  We
observe that the type of IS-CAPTURE-P is described by the theorem 
(OR (EQUAL (IS-CAPTURE-P M POS) T) (EQUAL (IS-CAPTURE-P M POS) NIL)).
We used primitive type reasoning.

Computing the guard conjecture for IS-CAPTURE-P....

The non-trivial part of the guard conjecture for IS-CAPTURE-P is

Goal
(IMPLIES (AND (POSITION-P POS) (MOVE-P M))
         (PIECE-P (ATTACKER POS))).
Goal'

Q.E.D.

That completes the proof of the guard theorem for IS-CAPTURE-P.  IS-CAPTURE-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN IS-CAPTURE-P ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:DEFINITION ATTACKER)
        (:DEFINITION MEMBER-EQUAL)
        (:DEFINITION MOVE-P)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION NATP)
        (:DEFINITION NOT)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-TYPE)
        (:DEFINITION PIECE-TYPE-P)
        (:DEFINITION POSITION-P)
        (:DEFINITION S

In [20]:
;; A move is a block if a non-king piece moves to interpose
;; (between attacker and king, blocking the attack line)
;; We abstract this: if it's not a king-move and not a capture, 
;; and it escapes check, it must be a block
(defun is-block-p (m pos)
  (declare (xargs :guard (and (move-p m) (position-p pos))))
  (and (not (is-king-move-p m pos))
       (not (is-capture-p m pos))))


Since IS-BLOCK-P is non-recursive, its admission is trivial.  We observe
that the type of IS-BLOCK-P is described by the theorem 
(OR (EQUAL (IS-BLOCK-P M POS) T) (EQUAL (IS-BLOCK-P M POS) NIL)). 

Computing the guard conjecture for IS-BLOCK-P....

The guard conjecture for IS-BLOCK-P is trivial to prove.  IS-BLOCK-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN IS-BLOCK-P ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 IS-BLOCK-P


In [21]:
(defthm move-classification-exhaustive
  (implies (and (move-p m)
                (position-p pos))
           (or (is-king-move-p m pos)
               (is-capture-p m pos)
               (is-block-p m pos)))
  :hints (("Goal" :in-theory (enable is-block-p))))


ACL2 Warning [Non-rec] in ( DEFTHM MOVE-CLASSIFICATION-EXHAUSTIVE ...):
A :REWRITE rule generated from MOVE-CLASSIFICATION-EXHAUSTIVE will
be triggered only by terms containing the function symbols IS-KING-MOVE-P,
IS-CAPTURE-P and IS-BLOCK-P, which have non-recursive definitions.
Unless these definitions are disabled, this rule is unlikely ever to
be used.

Goal'

Q.E.D.

The storage of MOVE-CLASSIFICATION-EXHAUSTIVE depends upon the :type-
prescription rules IS-BLOCK-P, IS-CAPTURE-P and IS-KING-MOVE-P.

Summary
Form:  ( DEFTHM MOVE-CLASSIFICATION-EXHAUSTIVE ...)
Rules: ((:DEFINITION ATTACKER)
        (:DEFINITION IS-BLOCK-P)
        (:DEFINITION IS-CAPTURE-P)
        (:DEFINITION IS-KING-MOVE-P)
        (:DEFINITION MEMBER-EQUAL)
        (:DEFINITION MOVE-DEST)
        (:DEFINITION MOVE-P)
        (:DEFINITION MOVE-PIECE)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION NATP)
        (:DEFINITION OUR-KING)
        (:DEFINITION PIECE-P)
        (:DEFINITION PIECE-SQUARE)
       

In [22]:
(defthm block-excludes-king-move
  (implies (and (move-p m)
                (position-p pos)
                (is-block-p m pos))
           (not (is-king-move-p m pos)))
  :hints (("Goal" :in-theory (enable is-block-p))))


ACL2 Warning [Non-rec] in ( DEFTHM BLOCK-EXCLUDES-KING-MOVE ...): 
A :REWRITE rule generated from BLOCK-EXCLUDES-KING-MOVE will be triggered
only by terms containing the function symbol IS-KING-MOVE-P, which
has a non-recursive definition.  Unless this definition is disabled,
this rule is unlikely ever to be used.


ACL2 Warning [Subsume] in ( DEFTHM BLOCK-EXCLUDES-KING-MOVE ...): 
The previously added rule IS-KING-MOVE-P subsumes a newly proposed
:REWRITE rule generated from BLOCK-EXCLUDES-KING-MOVE, in the sense
that the old rule rewrites a more general target.  Because the new
rule will be tried first, it may nonetheless find application.


Q.E.D.

Summary
Form:  ( DEFTHM BLOCK-EXCLUDES-KING-MOVE ...)
Rules: ((:DEFINITION ATTACKER)
        (:DEFINITION IS-BLOCK-P)
        (:DEFINITION IS-CAPTURE-P)
        (:DEFINITION IS-KING-MOVE-P)
        (:DEFINITION MOVE-DEST)
        (:DEFINITION MOVE-P)
        (:DEFINITION MOVE-PIECE)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION 

In [23]:
(defthm block-excludes-capture
  (implies (and (move-p m)
                (position-p pos)
                (is-block-p m pos))
           (not (is-capture-p m pos)))
  :hints (("Goal" :in-theory (enable is-block-p))))


ACL2 Warning [Non-rec] in ( DEFTHM BLOCK-EXCLUDES-CAPTURE ...):  A
:REWRITE rule generated from BLOCK-EXCLUDES-CAPTURE will be triggered
only by terms containing the function symbol IS-CAPTURE-P, which has
a non-recursive definition.  Unless this definition is disabled, this
rule is unlikely ever to be used.


ACL2 Warning [Subsume] in ( DEFTHM BLOCK-EXCLUDES-CAPTURE ...):  The
previously added rule IS-CAPTURE-P subsumes a newly proposed :REWRITE
rule generated from BLOCK-EXCLUDES-CAPTURE, in the sense that the old
rule rewrites a more general target.  Because the new rule will be
tried first, it may nonetheless find application.


Q.E.D.

Summary
Form:  ( DEFTHM BLOCK-EXCLUDES-CAPTURE ...)
Rules: ((:DEFINITION ATTACKER)
        (:DEFINITION IS-BLOCK-P)
        (:DEFINITION IS-CAPTURE-P)
        (:DEFINITION IS-KING-MOVE-P)
        (:DEFINITION MOVE-DEST)
        (:DEFINITION MOVE-P)
        (:DEFINITION MOVE-PIECE)
        (:DEFINITION MOVE-TYPE-P)
        (:DEFINITION NATP)
        

In [24]:
(encapsulate
  (((escapes-check * *) => *))
  
  ;; Witness function - always returns nil (trivially satisfies constraint)
  (local (defun escapes-check (m pos)
           (declare (ignore m pos))
           nil))
  
  ;; THE KEY CONSTRAINT: A move escapes check only if it's one of the three types
  ;; 
  ;; In a full model, we would prove this from first principles:
  ;; - King-move: king is on new square, may not be attacked there
  ;; - Block: piece interposes, breaking attack line
  ;; - Capture: attacker removed, attack eliminated
  (defthm escape-requires-valid-type
    (implies (and (move-p m)
                  (position-p pos)
                  (in-check-p pos)
                  (escapes-check m pos))
             (or (is-king-move-p m pos)
                 (is-capture-p m pos)
                 (is-block-p m pos)))))


To verify that the two encapsulated events correctly extend the current
theory we will evaluate them.  The theory thus constructed is only
ephemeral.

Encapsulated Events:


ACL2 !>>(LOCAL (DEFUN ESCAPES-CHECK (M POS)
                 (DECLARE (IGNORE M POS))
                 NIL))

Since ESCAPES-CHECK is non-recursive, its admission is trivial.  We
observe that the type of ESCAPES-CHECK is described by the theorem
(EQUAL (ESCAPES-CHECK M POS) NIL).  

Summary
Form:  ( DEFUN ESCAPES-CHECK ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
ESCAPES-CHECK


ACL2 !>>(DEFTHM ESCAPE-REQUIRES-VALID-TYPE
          (IMPLIES (AND (MOVE-P M)
                        (POSITION-P POS)
                        (IN-CHECK-P POS)
                        (ESCAPES-CHECK M POS))
                   (OR (IS-KING-MOVE-P M POS)
                       (IS-CAPTURE-P M POS)
                       (IS-BLOCK-P M POS))))

ACL2 Warning [Non-rec] in ( DEFTHM ESCAPE-REQUIRES-VALID-TYPE ...):
A 

In [25]:
(defthm three-ways-out-of-check
  (implies (and (move-p m)
                (position-p pos)
                (in-check-p pos)
                (escapes-check m pos))
           (or (is-king-move-p m pos)
               (is-capture-p m pos)
               (is-block-p m pos)))
  :hints (("Goal" :use escape-requires-valid-type)))


ACL2 Warning [Non-rec] in ( DEFTHM THREE-WAYS-OUT-OF-CHECK ...):  A
:REWRITE rule generated from THREE-WAYS-OUT-OF-CHECK will be triggered
only by terms containing the function symbols IS-KING-MOVE-P, IS-CAPTURE-P
and IS-BLOCK-P, which have non-recursive definitions.  Unless these
definitions are disabled, this rule is unlikely ever to be used.


ACL2 Warning [Subsume] in ( DEFTHM THREE-WAYS-OUT-OF-CHECK ...):  A
newly proposed :REWRITE rule generated from THREE-WAYS-OUT-OF-CHECK
probably subsumes the previously added :REWRITE rule 
ESCAPE-REQUIRES-VALID-TYPE, in the sense that the new rule will now
probably be applied whenever the old rule would have been.


ACL2 Warning [Subsume] in ( DEFTHM THREE-WAYS-OUT-OF-CHECK ...):  The
previously added rules ESCAPE-REQUIRES-VALID-TYPE and 
MOVE-CLASSIFICATION-EXHAUSTIVE subsume a newly proposed :REWRITE rule
generated from THREE-WAYS-OUT-OF-CHECK, in the sense that the old rules
rewrite more general targets.  Because the new rule will be trie

In [26]:
;; If there's no escape, it's checkmate (abstract definition)
(defun checkmate-p (pos)
  (declare (xargs :guard (position-p pos)))
  ;; Checkmate: in check and no move escapes
  ;; (Abstractly represented - would need move generation in full model)
  (and (in-check-p pos)
       t)) ; placeholder - full model would enumerate moves


Since CHECKMATE-P is non-recursive, its admission is trivial.  We observe
that the type of CHECKMATE-P is described by the theorem 
(OR (EQUAL (CHECKMATE-P POS) T) (EQUAL (CHECKMATE-P POS) NIL)).  

Computing the guard conjecture for CHECKMATE-P....

The guard conjecture for CHECKMATE-P is trivial to prove.  CHECKMATE-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN CHECKMATE-P ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHECKMATE-P
